In [ ]:
import json
import pandas as pd
# Opening JSON file
f = open('/home/ferdinand_t/data/CMS_2010_to_June_2022_ENGLISH.json')
# returns JSON object as
# a dictionary
data = json.load(f)
df = pd.DataFrame.from_dict(data)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [ ]:
# df['firstKeyword'] = df['keywords'].apply(lambda x: x[0]['name'] if len(x) != 0 else None)
# #df['secondKeyword'] = df['keywords'].apply(lambda x: x[1]['name'] if len(x) > 1 else None)
# #df['thirdKeyword'] = df['keywords'].apply(lambda x: x[2]['name'] if len(x) > 2 else None)
# #df['fourthKeyword'] = df['keywords'].apply(lambda x: x[3]['name'] if len(x) > 3 else None)

df['cleanFocusCategory'] = df['thematicFocusCategory'].apply(lambda x: x['name'] if x is not None else x)

# #df = df[['firstKeyword', 'secondKeyword', 'thirdKeyword', 'fourthKeyword', 'thematicFocusCategory', 'cleanFocusCategory']]

df = df[['firstKeyword', 'keywordStrings', 'cleanFocusCategory']]

df_clean = df.copy()
df_clean = df_clean.dropna()
# df_clean['cleanKeywordStrings'] = [' '.join(map(str, l)) for l in df_clean['keywordStrings']]

In [ ]:
vector_size_n_w2v = 5

w2v_model = Word2Vec(vector_size=vector_size_n_w2v,
                     window=3,
                     min_count=1,
                     sg=0) # 0=CBOW, 1=Skip-gram

print(w2v_model)

In [ ]:
w2v_model.build_vocab(df_clean['keywordStrings'])
print(w2v_model)

In [ ]:
w2v_model.train(df_clean['keywordStrings'], 
                total_examples=w2v_model.corpus_count, 
                epochs=5)

In [ ]:
vector_size_n_w2v = 5

w2v_model = Word2Vec(df_clean['keywordStrings'],
                     vector_size=vector_size_n_w2v,
                     window=3,
                     min_count=1,
                     sg=0, # 0=CBOW, 1=Skip-gram
                     epochs=5)

print(w2v_model)

In [ ]:
w2v_model.save("word2vec/word2vec_model")

pk.dump(vector_size_n_w2v, open('word2vec/vector_size_w2v_metric.pkl', 'wb'))

In [ ]:
words = set(w2v_model.wv.index_to_key )
df_clean['Text_vect'] = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in df_clean['keywordStrings']])

In [ ]:
text_vect_avg = []
for v in df_clean['Text_vect']:
    if v.size:
        text_vect_avg.append(v.mean(axis=0))
    else:
        text_vect_avg.append(np.zeros(vector_size_n_w2v, dtype=float)) # the same vector size must be used here as for model training
        
        
df_clean['Text_vect_avg'] = text_vect_avg
df_clean


In [ ]:
df_Machine_Learning = pd.DataFrame(text_vect_avg)

In [ ]:
df_Machine_Learning.columns = ['Element_' + str(i+1) for i in range(0, df_Machine_Learning.shape[1])]
df_Machine_Learning

In [ ]:
final_df = pd.concat([df_clean[['cleanFocusCategory', 'keywordStrings']], df_Machine_Learning], axis=1, sort=False)
final_df

In [ ]:
clf = SVC(kernel='linear')
clf.fit(df_Machine_Learning, final_df['cleanFocusCategory'])

In [ ]:
pk.dump(clf, open('clf_model.pkl', 'wb'))

In [ ]:
w2v_model_reloaded = Word2Vec.load("word2vec/word2vec_model")
vector_size_n_reloaded = pk.load(open("word2vec/vector_size_w2v_metric.pkl",'rb'))

print(w2v_model_reloaded)
print(vector_size_n_reloaded)